<a href="https://colab.research.google.com/github/dragonsan17/crime_det/blob/master/crime_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import nltk 
import string
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
from zipfile import ZipFile 
file_name = "/content/gdrive/My Drive/filtered_tweets_JK.json.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    zip.printdir() 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!')
data_path = '/content/filtered_tweets_JK.json'
f = open(data_path,'r')
tweets = json.load(f)
print(type(tweets))

File Name                                             Modified             Size
filtered_tweets_JK.json                        2019-07-23 11:44:12   1197687280
Extracting all the files now...
Done!
<class 'list'>


In [27]:
eng_tweets = []
for tweet in tweets:
  if(tweet['lang']=="en"):
    eng_tweets.append(tweet['text'])
print("appended ",len(eng_tweets)," tweets")

appended  101743  tweets


In [0]:
#Block : clean text, return tokenized text
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
ps = nltk.PorterStemmer()
stopword = nltk.corpus.stopwords.words('english')
def clean_token(text):
    #tokenize
    text = re.split('\W+', text)
    #remove stop words
    text = [word for word in text if word not in stopword]
    text = [word for word in text if word is not '']
    #stemming
    text = [ps.stem(word) for word in text]
    return text

def clean_text(eng_tweets):
  tokens = []
  for i in range(len(eng_tweets)):
      #remove handles, urls, RTs, emojis, hashtags
      original = eng_tweets[i]
      eng_tweets[i] = " ".join([word for word in eng_tweets[i].split()
                                  if 'http' not in word and '@' not in word and '<' not in word and '#' not in word])
      eng_tweets[i] = eng_tweets[i].encode('ascii', 'ignore').decode('ascii')
      #remove punctuations
      for punct in '.,"':
        eng_tweets[i] = eng_tweets[i].replace(punct, ' ')
      eng_tweets[i] = remove_punct(eng_tweets[i])
      #clean token
      tokens.append((clean_token(eng_tweets[i]),original))
  return tokens

tokens = clean_text(eng_tweets)

In [0]:
crimes = ['hostage','murder','robbery','con','fraud','kidnap','rape','terrorism','terror','crime','protest','pelt','arson']
crime_dict = {}
for crime in crimes:
  crime_dict[(crime,1)] = []
  crime_dict[(crime,-1)] = []

In [0]:
#input +1 or -1 for each tweet
# cnt=0
for token,tweet in tokens[:50000]:
  # if any(word in crimes for word in token):
  #   cnt+=1
  for word in token:
    if word in crimes:
      print("Crime : ",word," = \n",tweet)
      sign_of_crime = int(input())
      crime_dict[(word,sign_of_crime)].append((token,tweet))
# print(cnt)
# print(crime_dict)

In [0]:
#saves crime keyword, nature of crime (positive or negative), tokens, and original tweet text in csv

import csv
with open('/content/gdrive/My Drive/final_tweet_data.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for (crime,sign), values in crime_dict.items():
      for tokens,original_tweet in values:
       writer.writerow([crime,sign,tokens,original_tweet])